<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install
!pip install pdfminer.six
!pip install spacy

     |████████████████████████████████| 5.6MB 2.9MB/s 
     |████████████████████████████████| 13.7MB 44.4MB/s 


In [3]:
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 14.7MB 4.0MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=8eff767296727f3d64050bb422a4ae12f2988072f33a199db5493fe831cbd894
  Stored in directory: /tmp/pip-ephem-wheel-cache-77g3zph9/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
     |████████████████████████████████| 88.6MB 68.1MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=bfa23a5cb6fef03f2a7bccfd76cc8654ef840d37d55e61b365c666ddbcd4c854
  Stored in directory: /tmp/pip-ephem-wheel-cache-5e7yf7up/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [5]:
from google.colab import files

uploaded = files.upload()


Saving dataset.json to dataset.json


In [31]:
ls

dataset.json  sample_data/


In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [0]:
pdfs = []
for f in uploaded.keys():
  try:
    text=extract_text_from_pdf(f)
    if "Déclaration de franchissement de seuils" in text:
      pdfs.append({
          "file": f,
          "text": text,
      })
  except:
    pass

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
import json
with open('dataset.json') as f:
  pdfs = json.load(f)

In [0]:
ANNOTATION_START = 60

In [0]:
import IPython
from google.colab import output
import os

class AnnotWidget():
  def __init__(self, to_annot, start=0, model=None):
    self.to_annot = to_annot
    self.cur = start
    self.model = model
    self.colors_labels = [
        ("rgb(255, 0, 0)", "ISSUER", 'A'),
        ("rgb(255, 0, 255)", "FIRM", 'Z'),
        ("rgb(0, 255, 255)", "NUMBER", 'E'),
        ("rgb(160, 160, 255)", "PERCENT", 'R'),
        ("rgb(192, 192, 192)", "TYPE", 'T'),
        ("rgb(255, 255, 0)", "DIRECTION", 'S'),
        ("rgb(0, 255, 0)", "DATE", 'D')
      ]
    self.controls = ""
    for c, l, k in self.colors_labels:
      self.controls += '''<button id="key_'''+k+'''" style="background-color: '''+\
        c+''';" onclick='document.execCommand("backColor", false, "'''+c+'''")'>'''+l+" ("+k+")</button>"

  def text_entities_to_html(self):
    if 'entities' not in self.to_annot[self.cur] or len(self.to_annot[self.cur]['entities']) == 0:
      return self.to_annot[self.cur]["text"]
    res = ""
    offset = 0
    text = self.to_annot[self.cur]["text"]
    for ent_s, ent_e, label in self.to_annot[self.cur]['entities']:
      res += text[offset:ent_s]
      res += '''<span style="background-color: '''+ self.get_color(label) + '''">'''
      res += text[ent_s:ent_e]
      res += "</span>"
      offset = ent_e
    res += text[offset:]
    return res

  def run(self):
    display(IPython.display.HTML('''
    <p>'''+str(self.cur) + "/" + str(len(self.to_annot)) + '''</p>
    <div id="ed" contenteditable="true">'''+self.text_entities_to_html()+'''</div>
    <fieldset>
    '''+ self.controls +'''
      <button id=""key_Q" onclick="document.execCommand('removeFormat', false, null)">remove (Q)</button>
      <button onclick="google.colab.kernel.invokeFunction('notebook.Prediction');">pred</button>
      <button id='prev'>prev</button>
      <button id='next'>next</button>
    </fieldset>
    <script>
      function change(dir) {
        children = document.querySelector("#ed").childNodes;
        res = [];
        offset = 0;
        for (var i = 0; i < children.length; i++) {
          next_offset = offset + children[i].textContent.length;
          if (children[i].nodeType != Node.TEXT_NODE) {
            rgb = children[i].style.backgroundColor
            res.push([offset, next_offset, rgb])
          }
          offset = next_offset;
        }
        google.colab.kernel.invokeFunction('notebook.SaveEntities', [res, dir]);
        document.querySelector('#ed').innerHTML = next_text;
      };
      document.querySelector('#prev').onclick = () => {
        change(-1);
      };
      document.querySelector('#next').onclick = () => {
        change(1);
      };
      document.querySelector('#ed').onkeydown = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeypress = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeyup = (ev) => {
        ev.preventDefault();
        button_id = "#key_" + String.fromCharCode(ev.keyCode);
        button = document.querySelector(button_id)
        if (button) {
          button.click();
        }
      };
    </script>
    '''))
    output.register_callback('notebook.SaveEntities', self.save_entities)
    output.register_callback('notebook.Console', self.console)
    output.register_callback('notebook.Prediction', self.prediction)

  def get_color(self, label):
    for c, l, _ in self.colors_labels:
        if l == label:
          return c

  def get_label(self, color):
    for c, l, _ in self.colors_labels:
        if c == color:
          return l

  def save_entities(self, res, inc=1):
    self.to_annot[self.cur]['entities'] = []
    for ent_s, ent_e, color in res:
      self.to_annot[self.cur]['entities'].append((ent_s, ent_e, self.get_label(color)))
    if len(self.to_annot[self.cur]['entities']) != 0:
      name, _ = os.path.splitext(self.to_annot[self.cur]['file'])
      with open(name+'.json', 'w+') as f:
        json.dump(self.to_annot[self.cur], f)
    print("hello")
    output.clear()
    if (self.cur < len(self.to_annot) and inc == 1) or (self.cur > 0 and inc == -1):
      self.cur += inc
    self.run()

  def console(self, s):
    print(s)

  def prediction(self):
    if (self.model is not None) and (('entities' not in self.to_annot[self.cur]) or (len(self.to_annot[self.cur]['entities']) == 0)):
      doc = self.model(self.to_annot[self.cur]["text"])
      self.to_annot[self.cur]['entities'] = []
      for ent in doc.ents:
        self.to_annot[self.cur]['entities'].append([ent.start_char, ent.end_char, ent.label_])
      output.clear()
      self.run()


w = AnnotWidget(pdfs, ANNOTATION_START, nlp) 

In [0]:
w.run()

In [0]:
import json
with open("dataset.json", 'w+') as f:
  json.dump(w.to_annot, f)

In [115]:
!zip dataset.zip *.json

updating: 0070C022_220C0700.json (deflated 66%)
updating: 0340C022_220C0430.json (deflated 60%)
updating: 0510C022_220C0150.json (deflated 51%)
updating: 0540C022_220C0450.json (deflated 62%)
updating: 0620C022_220C0260.json (deflated 50%)
updating: 0810C022_220C0180.json (deflated 69%)
updating: 0820C022_220C0280.json (deflated 60%)
updating: 0900C022_220C0090.json (deflated 74%)
updating: 0940C022_220C0490.json (deflated 57%)
updating: 0960C022_220C0690.json (deflated 69%)
updating: 0980C022_220C0890.json (deflated 54%)
updating: 1040C022_220C0401.json (deflated 61%)
updating: 1080C022_220C0801.json (deflated 51%)
updating: 1090C022_220C0901.json (deflated 67%)
updating: 1200C022_220C0021.json (deflated 57%)
updating: 1270C022_220C0721.json (deflated 64%)
updating: 1360C022_220C0631.json (deflated 65%)
updating: 1380C022_220C0831.json (deflated 55%)
updating: 1430C022_220C0341.json (deflated 67%)
updating: 1500C022_220C0051.json (deflated 59%)
updating: 1550C022_220C0551.json (deflat

In [0]:
from google.colab import files
files.download('dataset.zip') 

In [0]:
train_data = []
for x in w.to_annot:
  entities = [tuple(ent) for ent in  x["entities"]]
  train_data.append((x['text'], {'entities' : entities}))
labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE']
test_data = train_data[:20]
train_data = train_data[20:]

In [126]:
import random
import spacy
from spacy.util import minibatch, compounding
import fr_core_news_md

n_iter=40
nlp = fr_core_news_md.load()
ner = nlp.get_pipe("ner")
for label in labels:
  ner.add_label(label)
optimizer = nlp.resume_training()
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  # batch up the examples using spaCy's minibatch
  for itn in range(n_iter):
    random.shuffle(train_data)
    batches = minibatch(train_data, size=sizes)
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
    print("Losses", losses)
  score = nlp.evaluate(test_data)

Losses {'ner': 24153.311322573623}
Losses {'ner': 21956.77616430167}
Losses {'ner': 21095.319226328982}
Losses {'ner': 20915.89358788007}
Losses {'ner': 20341.8768022554}
Losses {'ner': 20376.460788355136}
Losses {'ner': 20473.29834162764}
Losses {'ner': 19986.079776601866}
Losses {'ner': 20032.236200794578}
Losses {'ner': 19993.376051470637}
Losses {'ner': 19884.37486579269}
Losses {'ner': 19876.99413263239}
Losses {'ner': 19670.586275116773}
Losses {'ner': 19888.504910320044}
Losses {'ner': 20032.260424580425}
Losses {'ner': 19838.610898643732}
Losses {'ner': 19535.02372944355}
Losses {'ner': 19414.968142852187}
Losses {'ner': 19859.80466152313}
Losses {'ner': 19817.922621201724}
Losses {'ner': 19697.961311161518}
Losses {'ner': 19763.562167316675}
Losses {'ner': 19933.950015082955}
Losses {'ner': 20020.320873498917}
Losses {'ner': 19875.48264312744}
Losses {'ner': 19750.648679971695}
Losses {'ner': 20029.822446227074}
Losses {'ner': 20609.802905127406}
Losses {'ner': 19863.536083094

In [128]:
print(score.scores)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 92.62295081967213, 'ents_r': 94.9579831932773, 'ents_f': 93.7759336099585, 'ents_per_type': {'ISSUER': {'p': 84.61538461538461, 'r': 100.0, 'f': 91.66666666666666}, 'NUMBER': {'p': 92.85714285714286, 'r': 100.0, 'f': 96.2962962962963}, 'FIRM': {'p': 100.0, 'r': 68.75, 'f': 81.48148148148148}, 'DIRECTION': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'PERCENT': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'TYPE': {'p': 93.10344827586206, 'r': 100.0, 'f': 96.42857142857143}, 'DATE': {'p': 90.47619047619048, 'r': 95.0, 'f': 92.68292682926828}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}


In [119]:
from spacy import displacy
doc = nlp(pdfs[0]['text'])
displacy.render(doc, style='ent', jupyter=True)

In [0]:
!pip install tabula-py

     |████████████████████████████████| 10.4MB 3.3MB/s 


In [0]:
from tabula import read_pdf

list_pdf = list(pdfs.keys())
df = read_pdf(list_pdf[0], pages='all')

Got stderr: Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [0]:
import pandas as pd
for d in df:
  if not d.empty:
    print(d)
    table_str = ""
    for c in d.columns:
      if not "Unnamed:" in c:
        table_str += c + " "
    for _, r in d.iterrows():
      for c in d.columns:
        if pd.notna(r[c]):
          table_str += r[c] + " "
    print(table_str)

                          Unnamed: 0  Unnamed: 1  ... % droits de vot Unnamed: 10
0        Merrill Lynch International  14 101 661  ...             NaN         NaN
1          BofA Securities Europe SA     276 021  ...             NaN         NaN
2              BofA Securities, Inc.         213  ...             NaN         NaN
3  Total Bank of America Corporation  14 377 895  ...             NaN         NaN

[4 rows x 15 columns]
Actions % capita Droits de vote % droits de vot Merrill Lynch International 14 101 661 5,49 14 101 661 5,47 BofA Securities Europe SA 276 021 0,11 276 021 0,11 BofA Securities, Inc. 213 ns 213 ns Total Bank of America Corporation 14 377 895 5,59 14 377 895 5,58 


In [0]:
import json
with open("texts.json", "w+") as f:
  json.dump(pdfs, f)


In [0]:
with open("texts.txt", "w+") as f:
  f.write(pdfs[list_pdf[0]]['text'])

In [24]:
import spacy
import fr_core_news_sm
import fr_core_news_md

#nlp = fr_core_news_sm.load()
#nlp = fr_core_news_md.load()

doc = nlp(train_data[0][0])
for sent in doc.sents:
    print(sent)



 220C0490-CH0308403085-FS0132             220C0490 CH0308403085-FS0132 6 février 2020   
Déclaration de franchissement de seuils (article L. 233
-7 du code de commerce)    GENEURO
SA  (Euronext Paris)    Par courrier reçu le 5 février 2020, complété par un courrier reçu le 6 février, la société Invesco Ltd.
(Invesco Head Quarters, Two Peachtree Pointe 1555 Peachtree St, Suite 1800 Atlanta, Georgie, Etats
-Unis), agissant pour le compte du fonds Invesco Advisers, Inc.
dont elle assure la gestion, a déclaré avoir franchi en hausse, le 4 février 2020, les seuils 5% du capital et des droits de vote de la société GENEURO SA et détenir, pour le compte dudit fonds, 1 661 017 actions GENEURO SA représentant autant de droits de vote, soit 8,07% du capital et des droits de vote de cette société1.  
Ce franchissement de seuils résulte de la souscription à l’augmentation de capital de la société GENEURO SA2.  
________                                                            1 Sur la base d’un c

In [0]:
for i in range(len(train_data[0][1]['entities'])):
  s, e, _ = train_data[0][1]['entities'][i]
  print(train_data[0][0][s:e])


21 février 2020
ALTRAN TECHNOLOGIES
20 février 2020
21 février
Bank of America Corporation
en hausse
17 février 2020
5%
capital
droits de vote
ALTRAN TECHNOLOGIES
14 377 895
actions
ALTRAN TECHNOLOGIES
5,59%
capital
5,58%
droits de vote
ALTRAN TECHNOLOGIES
ALTRAN TECHNOLOGIES
Merrill Lynch International
en hausse
Merrill Lynch International
13 784 025
actions
ALTRAN TECHNOLOGIES
Merrill Lynch International
BofA Securities Europe SA
274 482
actions
ALTRAN TECHNOLOGIES
BofA Securities Europe SA
Merrill Lynch International
17 749
actions
ALTRAN TECHNOLOGIES


In [60]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

In [0]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)
 

220C0170-FR0012407096-OP028-AS12-RO02            220C0170 FR0012407096-OP028-AS12-RO02 14 janvier 2020    NP 220C0170-FR0012407096-OP028-AS12-RO02
- Mise en œuvre du retrait obligatoire visant les actions de la société.   NP Mise
- Maintien de la suspension de la cotation des actions de la société.     NP Maintien
Euronext Paris NP Euronext
Le 13 janvier 2020 NP janvier
l’Autorité des marchés financiers NP Autorité
d’achat simplifiée NP achat
les actions ITS GROUP NP actions
la société ITS Participations NP société
actions ITS GROUP NP actions
de droits de vote NP droits
des droits de vote de cette société2 NP droits
cf NP cf
D&I 220C0149 en date du 13 janvier 2020).   NP D&I
Les actions ITS GROUP non présentées à l’offre par les actionnaires minoritaires NP actions
de 366 396 actions NP actions
483 droits de vote, NP droits
soit 4,65% du capital et 4,92% des droits de vote de cette société2 NP %
2- Par courrier NP 2-
de la société NP société
ITS Participations NP Participations
l’Auto

In [65]:
for entity in doc.ents:
    print(entity.start_char, entity.end_char, entity.label_)

72 86 DATE
198 223 ISSUER
267 281 DATE
382 391 DIRECTION
396 410 DATE
460 516 FIRM
520 523 PERCENT
527 534 TYPE
542 556 TYPE
571 594 ISSUER
637 647 NUMBER
648 655 TYPE
656 679 ISSUER
724 730 PERCENT
734 741 TYPE
749 763 TYPE
846 860 TYPE
861 864 ISSUER
882 895 NUMBER
943 961 NUMBER
1095 1099 ISSUER
1115 1129 DATE
1196 1210 ISSUER
1216 1239 ISSUER
1258 1261 ISSUER
1480 1483 ISSUER
1602 1605 ISSUER
1618 1621 PERCENT
1625 1632 TYPE
1640 1654 TYPE
1669 1721 ISSUER
1751 1765 ISSUER
1771 1794 ISSUER
1813 1816 ISSUER
1913 1916 ISSUER
1920 1934 ISSUER
1939 1953 DATE
2096 2131 FIRM
2159 2162 PERCENT
2170 2210 FIRM
2211 2214 PERCENT
2221 2236 FIRM
2330 2341 NUMBER
2342 2349 TYPE
2620 2634 DATE
2674 2677 ISSUER
2839 2862 ISSUER
2970 2993 ISSUER
3046 3069 ISSUER
3465 3488 ISSUER
3587 3614 FIRM
3672 3695 ISSUER
